In [1]:
import data_manager

In [4]:
DBdata = data_manager.read_DBdata()

client
get_database
SQL
Today: 2020-10-24
Last Data: 2020-09-29


In [5]:
DBdata

,date,open,high,low,close,volume
0,20190102,39400,39400,38550,38750,7847664
1,20190103,38300,38550,37450,37600,12471493
2,20190104,37450,37600,36850,37450,14108958
3,20190107,38000,38900,37800,38750,12748997
4,20190108,38000,39200,37950,38100,12756554
...,...,...,...,...,...,...
428,20200923,58400,58800,57400,58600,20111398
429,20200924,57700,58600,57600,57800,17564020
430,20200925,57700,58200,57700,57900,11444683
431,20200928,58300,58800,57900,58200,12614080


In [6]:
chart, train = data_manager.load_data(DBdata)

차트데이터: Index(['date', 'open', 'high', 'low', 'close', 'volume'], dtype='object') 
트레이닝데이터 Index(['open_lastclose_ratio', 'high_close_ratio', 'low_close_ratio',
       'close_lastclose_ratio', 'volume_lastvolume_ratio', 'close_ma5_ratio',
       'volume_ma5_ratio', 'close_ma10_ratio', 'volume_ma10_ratio',
       'close_ma20_ratio', 'volume_ma20_ratio', 'close_ma60_ratio',
       'volume_ma60_ratio', 'close_ma120_ratio', 'volume_ma120_ratio'],
      dtype='object')


In [7]:
chart.head(3)

,date,open,high,low,close,volume
0,20190102,39400,39400,38550,38750,7847664
1,20190103,38300,38550,37450,37600,12471493
2,20190104,37450,37600,36850,37450,14108958


In [13]:
train.isna().sum()

open_lastclose_ratio         0
high_close_ratio             0
low_close_ratio              0
close_lastclose_ratio        0
volume_lastvolume_ratio      0
close_ma5_ratio              4
volume_ma5_ratio             4
close_ma10_ratio             9
volume_ma10_ratio            9
close_ma20_ratio            19
volume_ma20_ratio           19
close_ma60_ratio            59
volume_ma60_ratio           59
close_ma120_ratio          119
volume_ma120_ratio         119
dtype: int64

In [14]:
data_manager.preprocess(DBdata).isna().sum()

date                         0
open                         0
high                         0
low                          0
close                        0
volume                       0
close_ma5                    4
volume_ma5                   4
close_ma5_ratio              4
volume_ma5_ratio             4
close_ma10                   9
volume_ma10                  9
close_ma10_ratio             9
volume_ma10_ratio            9
close_ma20                  19
volume_ma20                 19
close_ma20_ratio            19
volume_ma20_ratio           19
close_ma60                  59
volume_ma60                 59
close_ma60_ratio            59
volume_ma60_ratio           59
close_ma120                119
volume_ma120               119
close_ma120_ratio          119
volume_ma120_ratio         119
open_lastclose_ratio         0
high_close_ratio             0
low_close_ratio              0
close_lastclose_ratio        0
volume_lastvolume_ratio      0
dtype: int64

### 결측값 있는 것으로 보이는데 이동평균때문에 그런것. 문제없음

In [13]:
import pandas as pd
import numpy as np
from azure.cosmos import CosmosClient
from datetime import date


COLUMNS_CHART_DATA = ['date', 'open', 'high', 'low', 'close', 'volume']

COLUMNS_TRAINING_DATA_V1 = [
    'open_lastclose_ratio', 'high_close_ratio', 'low_close_ratio',
    'close_lastclose_ratio', 'volume_lastvolume_ratio',
    'close_ma5_ratio', 'volume_ma5_ratio',
    'close_ma10_ratio', 'volume_ma10_ratio',
    'close_ma20_ratio', 'volume_ma20_ratio',
    'close_ma60_ratio', 'volume_ma60_ratio',
    'close_ma120_ratio', 'volume_ma120_ratio',
]

COLUMNS_TRAINING_DATA_V1_RICH = [
    'open_lastclose_ratio', 'high_close_ratio', 'low_close_ratio',
    'close_lastclose_ratio', 'volume_lastvolume_ratio',
    'close_ma5_ratio', 'volume_ma5_ratio',
    'close_ma10_ratio', 'volume_ma10_ratio',
    'close_ma20_ratio', 'volume_ma20_ratio',
    'close_ma60_ratio', 'volume_ma60_ratio',
    'close_ma120_ratio', 'volume_ma120_ratio',
    'inst_lastinst_ratio', 'frgn_lastfrgn_ratio',
    'inst_ma5_ratio', 'frgn_ma5_ratio',
    'inst_ma10_ratio', 'frgn_ma10_ratio',
    'inst_ma20_ratio', 'frgn_ma20_ratio',
    'inst_ma60_ratio', 'frgn_ma60_ratio',
    'inst_ma120_ratio', 'frgn_ma120_ratio',
]

COLUMNS_TRAINING_DATA_V2 = [
    'per', 'pbr', 'roe',
    'open_lastclose_ratio', 'high_close_ratio', 'low_close_ratio',
    'close_lastclose_ratio', 'volume_lastvolume_ratio',
    'close_ma5_ratio', 'volume_ma5_ratio',
    'close_ma10_ratio', 'volume_ma10_ratio',
    'close_ma20_ratio', 'volume_ma20_ratio',
    'close_ma60_ratio', 'volume_ma60_ratio',
    'close_ma120_ratio', 'volume_ma120_ratio',
    'market_kospi_ma5_ratio', 'market_kospi_ma20_ratio', 
    'market_kospi_ma60_ratio', 'market_kospi_ma120_ratio', 
    'bond_k3y_ma5_ratio', 'bond_k3y_ma20_ratio', 
    'bond_k3y_ma60_ratio', 'bond_k3y_ma120_ratio'
]


def preprocess(data, ver='v1'):
    windows = [5, 10, 20, 60, 120]
    for window in windows:
        data['close_ma{}'.format(window)] = \
            data['close'].rolling(window).mean()
        data['volume_ma{}'.format(window)] = \
            data['volume'].rolling(window).mean()
        data['close_ma%d_ratio' % window] = \
            (data['close'] - data['close_ma%d' % window]) \
            / data['close_ma%d' % window]
        data['volume_ma%d_ratio' % window] = \
            (data['volume'] - data['volume_ma%d' % window]) \
            / data['volume_ma%d' % window]
            
        if ver == 'v1.rich':
            data['inst_ma{}'.format(window)] = \
                data['close'].rolling(window).mean()
            data['frgn_ma{}'.format(window)] = \
                data['volume'].rolling(window).mean()
            data['inst_ma%d_ratio' % window] = \
                (data['close'] - data['inst_ma%d' % window]) \
                / data['inst_ma%d' % window]
            data['frgn_ma%d_ratio' % window] = \
                (data['volume'] - data['frgn_ma%d' % window]) \
                / data['frgn_ma%d' % window]

    data['open_lastclose_ratio'] = np.zeros(len(data))
    data.loc[1:, 'open_lastclose_ratio'] = \
        (data['open'][1:].values - data['close'][:-1].values) \
        / data['close'][:-1].values
    data['high_close_ratio'] = \
        (data['high'].values - data['close'].values) \
        / data['close'].values
    data['low_close_ratio'] = \
        (data['low'].values - data['close'].values) \
        / data['close'].values
    data['close_lastclose_ratio'] = np.zeros(len(data))
    data.loc[1:, 'close_lastclose_ratio'] = \
        (data['close'][1:].values - data['close'][:-1].values) \
        / data['close'][:-1].values
    data['volume_lastvolume_ratio'] = np.zeros(len(data))
    data.loc[1:, 'volume_lastvolume_ratio'] = \
        (data['volume'][1:].values - data['volume'][:-1].values) \
        / data['volume'][:-1] \
            .replace(to_replace=0, method='ffill') \
            .replace(to_replace=0, method='bfill').values

    if ver == 'v1.rich':
        data['inst_lastinst_ratio'] = np.zeros(len(data))
        data.loc[1:, 'inst_lastinst_ratio'] = \
            (data['inst'][1:].values - data['inst'][:-1].values) \
            / data['inst'][:-1] \
                .replace(to_replace=0, method='ffill') \
                .replace(to_replace=0, method='bfill').values
        data['frgn_lastfrgn_ratio'] = np.zeros(len(data))
        data.loc[1:, 'frgn_lastfrgn_ratio'] = \
            (data['frgn'][1:].values - data['frgn'][:-1].values) \
            / data['frgn'][:-1] \
                .replace(to_replace=0, method='ffill') \
                .replace(to_replace=0, method='bfill').values

    return data

def load_data(fpath, date_from, date_to, ver='v2'):
    header = None if ver == 'v1' else 0
    data = pd.read_csv(fpath, thousands=',', header=header, 
        converters={'date': lambda x: str(x)})

    if ver == 'v1':
        data.columns = ['date', 'open', 'high', 'low', 'close', 'volume']

    # 날짜 오름차순 정렬
    data = data.sort_values(by='date').reset_index()

    # 데이터 전처리
    data = preprocess(data)
    
    # 기간 필터링
    data['date'] = data['date'].str.replace('-', '')
    data = data[(data['date'] >= date_from) & (data['date'] <= date_to)]
    data = data.dropna()

    # 차트 데이터 분리
    chart_data = data[COLUMNS_CHART_DATA]

    # 학습 데이터 분리
    training_data = None
    if ver == 'v1':
        training_data = data[COLUMNS_TRAINING_DATA_V1]
    elif ver == 'v1.rich':
        training_data = data[COLUMNS_TRAINING_DATA_V1_RICH]
    elif ver == 'v2':
        data.loc[:, ['per', 'pbr', 'roe']] = \
            data[['per', 'pbr', 'roe']].apply(lambda x: x / 100)
        training_data = data[COLUMNS_TRAINING_DATA_V2]
        training_data = training_data.apply(np.tanh)
    else:
        raise Exception('Invalid version.')
    
    return chart_data, training_data


In [16]:
data = load_data('./data/v1/005930.csv', '20180101', '20181231', ver='v1')

In [19]:
chart, training = data

In [22]:
chart

,date,open,high,low,close,volume
8794,20180102,2569000,2570000,2539000,2551000,169485
8795,20180103,2627000,2628000,2571000,2581000,200270
8796,20180104,2606000,2609000,2532000,2554000,233909
8797,20180105,2565000,2606000,2560000,2606000,189623
8798,20180108,2620000,2626000,2575000,2601000,167673
8799,20180109,2573000,2586000,2499000,2520000,360272
8800,20180110,2525000,2526000,2432000,2442000,371336
8801,20180111,2410000,2463000,2401000,2412000,502476
8802,20180112,2412000,2424000,2338000,2410000,545409
8803,20180115,2440000,2449000,2396000,2427000,201920
